In [48]:
import numpy as np

from qiskit import BasicAer, ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit.tools.visualization import plot_histogram
from qiskit.aqua import Operator, run_algorithm
from qiskit.aqua.translators.ising import max_cut
from qiskit.aqua.algorithms import QAOA, VQE
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua import QuantumInstance
from qiskit.quantum_info import Pauli

from qiskit.aqua.components.variational_forms import RY

from pyqubo import Binary

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

In [18]:
## EXAMPLE Problem -> Replace with general form
binArr = []
variable = []
index = 0
for i in range(0,2):
    binArr.append(Binary('s'+str(i+1)))
    variable.append('s' + str(i+1))
#s1,s2,s3,s4,s5 = binArr[0], binArr[1], binArr[2], binArr[3], binArr[4]
H =  s2+ s1
H = H * H

In [ ]:
def get_ising_opt_qubitops(model, variables):
    
    num_variables = len(variables)
    var_to_int = dict(zip(variables, range(num_variables)))

    linear, quadratic, offset = model.compile().to_ising()
    
    pauli_list = []
    for (i,j),weight in quadratic.items():
        xs = np.zeros(num_variables, dtype=np.bool)
        zs = np.zeros(num_variables, dtype=np.bool)
        zs[var_to_int[i]] = True
        zs[var_to_int[j]] = True
        pauli_list.append([weight, Pauli(zs, xs)])
    
    for (i,weight) in linear.items():
        xs = np.zeros(num_variables, dtype=np.bool)
        zs = np.zeros(num_variables, dtype=np.bool)
        zs[var_to_int[i]] = True
        pauli_list.append([weight, Pauli(zs, xs)])
        
    return Operator(paulis=pauli_list), offset

In [50]:
def run_IBM(H=H, backend = None, num_samples=100,qaoa_steps=1):
    variables = variable#H.variables()
    num_vars = len(variables)
    
    qubit_op, offset = get_ising_opt_qubitops(H,variables)
    
    if backend == None:
        backend = BasicAer.get_backend('statevector_simulator')
    quantum_instance = QuantumInstance(backend)
    
    
    spsa = SPSA(max_trials=300)
    qaoa = QAOA(qubit_op, spsa, qaoa_steps)    
    result = qaoa.run(quantum_instance)
    
    circ = qaoa.get_optimal_circuit()
    q = circ.qregs
    print(q)
    c = ClassicalRegister(num_vars)
    circ.measure(q,c)
    job=execute(circ,backend,shots=num_samples)
    return job.result().get_counts(circ)
    
    
    
   # results = []
    
    
    
#             # if the backend is a statevector, we get a statevector as the optimal vector
#             #  we want samples from this state
#             if qaoa._quantum_instance.is_statevector:
#                 opt_vector = qaoa.get_optimal_vector()
#                 for i in range(len(opt_vector)):
#                     opt_vector[i] = np.norm(opt_vector[i])**2
#                 for i in range(num_samples):
#                     p = np.random.random():
#                     j = -1
#                     while p > 0:
#                         j += 1
#                         p -= opt_vector[i]  

#             # otherwise we actually get samples for the optimal vector
#             else:
#                 for i in range(num_samples):
#                     results.append(qaoa.get_optimal_vector())    
    
    return result

In [51]:
result = run_IBM()

0.5
[QuantumRegister(2, 'q')]


QiskitError: "Type error handling [QuantumRegister(2, 'q')] (<class 'list'>)"

In [28]:
opt_vector result[0]

array([-0.69915972-0.24529751j,  0.41942417-0.19111815j,
        0.41942417-0.19111815j,  0.15961794+0.02532527j])

In [35]:
result.get_counts()

AttributeError: 'list' object has no attribute 'get_counts'

In [41]:
result

{'num_optimizer_evals': None,
 'min_val': 1.7824338993042506,
 'opt_params': array([-1.23223955e+15,  1.72966858e+15]),
 'eval_time': 27.80966877937317,
 'eval_count': 651,
 'energy': 1.7824338993042506,
 'eigvals': array([1.7824339]),
 'min_vector': array([ 0.41409144-0.78400652j,  0.12761463-0.22946802j,
         0.12761463-0.22946802j, -0.16563844-0.22032666j]),
 'eigvecs': array([[ 0.41409144-0.78400652j,  0.12761463-0.22946802j,
          0.12761463-0.22946802j, -0.16563844-0.22032666j]])}